# Day13_1: 분류 모델 (Classification Models)

## 학습 목표

**Part 1: 기본 분류기**
1. Logistic Regression의 원리와 적용
2. K-Nearest Neighbors (KNN) 이해하기
3. Naive Bayes 분류기 활용
4. Support Vector Machine (SVM) 기초
5. 기본 분류기 성능 비교

**Part 2: 트리 기반 모델**
1. Decision Tree Classifier 이해
2. Random Forest Classifier 활용
3. Gradient Boosting Classifier 적용
4. 트리 기반 모델 비교 분석

**Part 3: 앙상블 기법**
1. Voting Classifier (Hard/Soft)
2. Bagging 기법 이해
3. Boosting 기법 (AdaBoost, XGBoost)
4. Stacking 앙상블 구현

**Part 4: 불균형 데이터 처리**
1. 클래스 불균형 문제 이해
2. class_weight 조정 방법
3. SMOTE 오버샘플링 적용
4. Plotly로 ROC/PR 곡선 시각화

---

## 왜 이것을 배우나요?

| 개념 | 비즈니스 활용 | 실무 예시 |
|------|-------------|----------|
| 기본 분류기 | 빠른 프로토타이핑 | 스팸 필터, 고객 이탈 예측 |
| 트리 기반 모델 | 해석 가능한 예측 | 대출 승인, 보험 심사 |
| 앙상블 기법 | 최고 성능 달성 | Kaggle 대회, 프로덕션 모델 |
| 불균형 처리 | 희귀 이벤트 탐지 | 사기 탐지, 질병 진단 |

**분석가 관점**: Week 4 Kaggle 미니 대회의 핵심! 다양한 분류 알고리즘을 이해하고 앙상블로 성능을 극대화하며, 불균형 데이터(사기 탐지)를 다루는 실전 스킬을 익힙니다.

---

# Part 1: 기본 분류기

---

## 1.1 환경 설정 및 데이터 준비

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

# 분류 알고리즘
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, BaggingClassifier, AdaBoostClassifier,
    StackingClassifier
)

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# 실습 데이터: 신용카드 사기 탐지 시뮬레이션 데이터
np.random.seed(42)

# 정상 거래 (95%)
n_normal = 9500
normal_data = {
    'amount': np.random.exponential(scale=100, size=n_normal),
    'hour': np.random.randint(6, 23, n_normal),
    'distance_from_home': np.random.exponential(scale=10, size=n_normal),
    'ratio_to_median': np.random.uniform(0.5, 2.0, n_normal),
    'repeat_retailer': np.random.choice([0, 1], n_normal, p=[0.3, 0.7]),
    'used_chip': np.random.choice([0, 1], n_normal, p=[0.2, 0.8]),
    'used_pin': np.random.choice([0, 1], n_normal, p=[0.3, 0.7]),
    'online_order': np.random.choice([0, 1], n_normal, p=[0.6, 0.4]),
    'fraud': np.zeros(n_normal)
}

# 사기 거래 (5%) - 패턴이 다름
n_fraud = 500
fraud_data = {
    'amount': np.random.exponential(scale=500, size=n_fraud),  # 더 큰 금액
    'hour': np.random.choice([0, 1, 2, 3, 4, 5, 23], n_fraud),  # 야간 거래
    'distance_from_home': np.random.exponential(scale=100, size=n_fraud),  # 먼 거리
    'ratio_to_median': np.random.uniform(3.0, 10.0, n_fraud),  # 평소보다 큰 거래
    'repeat_retailer': np.random.choice([0, 1], n_fraud, p=[0.8, 0.2]),  # 새 판매자
    'used_chip': np.random.choice([0, 1], n_fraud, p=[0.7, 0.3]),  # 칩 미사용
    'used_pin': np.random.choice([0, 1], n_fraud, p=[0.8, 0.2]),  # PIN 미사용
    'online_order': np.random.choice([0, 1], n_fraud, p=[0.2, 0.8]),  # 온라인 거래
    'fraud': np.ones(n_fraud)
}

# 데이터프레임 생성
df_normal = pd.DataFrame(normal_data)
df_fraud = pd.DataFrame(fraud_data)
df = pd.concat([df_normal, df_fraud], ignore_index=True)

# 데이터 셔플
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"전체 데이터: {len(df)}건")
print(f"사기 비율: {df['fraud'].mean():.2%}")
print(f"\n데이터 미리보기:")
df.head()

In [ ]:
# 특성과 타겟 분리
feature_cols = ['amount', 'hour', 'distance_from_home', 'ratio_to_median',
                'repeat_retailer', 'used_chip', 'used_pin', 'online_order']

X = df[feature_cols]
y = df['fraud']

# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"훈련 데이터: {X_train.shape[0]}건")
print(f"테스트 데이터: {X_test.shape[0]}건")
print(f"훈련 데이터 사기 비율: {y_train.mean():.2%}")
print(f"테스트 데이터 사기 비율: {y_test.mean():.2%}")

---

## 1.2 Logistic Regression

### 원리
- 선형 결합 결과를 시그모이드 함수로 변환하여 확률 출력
- 이진 분류의 기본 베이스라인 모델
- 해석 가능성이 높음 (계수 = 특성 중요도)

In [ ]:
# Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# 예측
y_pred_lr = lr_model.predict(X_test_scaled)
y_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("=" * 50)
print("Logistic Regression 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_lr, target_names=['정상', '사기']))

# 계수 확인 (특성 중요도)
print("\n특성별 계수 (중요도):")
for feature, coef in sorted(zip(feature_cols, lr_model.coef_[0]), key=lambda x: abs(x[1]), reverse=True):
    print(f"  {feature}: {coef:.4f}")

---

## 1.3 K-Nearest Neighbors (KNN)

### 원리
- 가장 가까운 K개 이웃의 다수결로 분류
- 비모수적 방법 (데이터 분포 가정 없음)
- K 값에 따라 성능 변화

In [ ]:
# KNN (K=5)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# 예측
y_pred_knn = knn_model.predict(X_test_scaled)
y_proba_knn = knn_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("=" * 50)
print("K-Nearest Neighbors (K=5) 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_knn, target_names=['정상', '사기']))

In [ ]:
# K 값에 따른 성능 변화
k_values = range(1, 21, 2)
k_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='f1')
    k_scores.append(scores.mean())

# Plotly 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(k_values), y=k_scores,
    mode='lines+markers',
    name='F1 Score',
    marker=dict(size=10)
))

best_k = list(k_values)[np.argmax(k_scores)]
fig.add_vline(x=best_k, line_dash="dash", line_color="red",
              annotation_text=f"Best K={best_k}")

fig.update_layout(
    title="KNN: K 값에 따른 F1 Score",
    xaxis_title="K (이웃 수)",
    yaxis_title="F1 Score (CV)",
    template="plotly_white"
)
fig.show()

print(f"\n최적 K: {best_k} (F1: {max(k_scores):.4f})")

---

## 1.4 Naive Bayes

### 원리
- 베이즈 정리 기반, 특성 간 독립 가정
- 매우 빠른 학습/예측
- 텍스트 분류에 특히 효과적

In [ ]:
# Gaussian Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)

# 예측
y_pred_nb = nb_model.predict(X_test_scaled)
y_proba_nb = nb_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("=" * 50)
print("Naive Bayes 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_nb, target_names=['정상', '사기']))

---

## 1.5 Support Vector Machine (SVM)

### 원리
- 클래스 간 마진을 최대화하는 결정 경계 탐색
- 커널 트릭으로 비선형 분류 가능
- 고차원 데이터에 효과적

In [ ]:
# SVM with RBF kernel
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)

# 예측
y_pred_svm = svm_model.predict(X_test_scaled)
y_proba_svm = svm_model.predict_proba(X_test_scaled)[:, 1]

# 평가
print("=" * 50)
print("SVM (RBF Kernel) 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_svm, target_names=['정상', '사기']))

---

## 1.6 기본 분류기 비교

In [ ]:
# 기본 분류기 성능 비교
basic_models = {
    'Logistic Regression': (y_pred_lr, y_proba_lr),
    'KNN': (y_pred_knn, y_proba_knn),
    'Naive Bayes': (y_pred_nb, y_proba_nb),
    'SVM': (y_pred_svm, y_proba_svm)
}

results = []
for name, (y_pred, y_proba) in basic_models.items():
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'AUC': auc(*roc_curve(y_test, y_proba)[:2])
    })

df_basic_results = pd.DataFrame(results)
print("기본 분류기 성능 비교:")
print(df_basic_results.round(4).to_string(index=False))

In [ ]:
# Plotly로 성능 비교 시각화
fig = go.Figure()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']
colors = px.colors.qualitative.Set2

for i, metric in enumerate(metrics):
    fig.add_trace(go.Bar(
        name=metric,
        x=df_basic_results['Model'],
        y=df_basic_results[metric],
        marker_color=colors[i],
        text=df_basic_results[metric].round(3),
        textposition='auto'
    ))

fig.update_layout(
    title="기본 분류기 성능 비교",
    xaxis_title="Model",
    yaxis_title="Score",
    barmode='group',
    template="plotly_white",
    legend_title="Metric"
)
fig.show()

---

# Part 2: 트리 기반 모델

---

## 2.1 Decision Tree Classifier

### 원리
- 특성 값으로 데이터를 분할하는 규칙 트리 생성
- 높은 해석 가능성 (시각화 가능)
- 과적합 경향 (가지치기 필요)

In [ ]:
# Decision Tree
dt_model = DecisionTreeClassifier(
    max_depth=5,           # 과적합 방지
    min_samples_split=20,  # 분할 최소 샘플
    random_state=42
)
dt_model.fit(X_train, y_train)  # 트리는 스케일링 불필요

# 예측
y_pred_dt = dt_model.predict(X_test)
y_proba_dt = dt_model.predict_proba(X_test)[:, 1]

# 평가
print("=" * 50)
print("Decision Tree 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_dt, target_names=['정상', '사기']))

# 특성 중요도
print("\n특성 중요도:")
for feature, importance in sorted(zip(feature_cols, dt_model.feature_importances_), 
                                   key=lambda x: x[1], reverse=True):
    print(f"  {feature}: {importance:.4f}")

---

## 2.2 Random Forest Classifier

### 원리
- 다수의 Decision Tree 앙상블 (Bagging)
- 각 트리는 부트스트랩 샘플 + 랜덤 특성 선택
- 과적합 방지, 높은 안정성

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,      # 트리 개수
    max_depth=10,          # 최대 깊이
    min_samples_split=10,  # 분할 최소 샘플
    n_jobs=-1,             # 병렬 처리
    random_state=42
)
rf_model.fit(X_train, y_train)

# 예측
y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# 평가
print("=" * 50)
print("Random Forest 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_rf, target_names=['정상', '사기']))

In [ ]:
# 특성 중요도 시각화 (Plotly)
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=True)

fig = go.Figure(go.Bar(
    x=feature_importance['importance'],
    y=feature_importance['feature'],
    orientation='h',
    marker_color='steelblue',
    text=feature_importance['importance'].round(3),
    textposition='auto'
))

fig.update_layout(
    title="Random Forest 특성 중요도",
    xaxis_title="Importance",
    yaxis_title="Feature",
    template="plotly_white"
)
fig.show()

---

## 2.3 Gradient Boosting Classifier

### 원리
- 순차적으로 트리를 추가하며 오차를 줄임
- 이전 트리의 잔차(residual)를 학습
- 높은 성능, 튜닝 필요

In [ ]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,      # 부스팅 라운드
    learning_rate=0.1,     # 학습률
    max_depth=5,           # 트리 깊이
    random_state=42
)
gb_model.fit(X_train, y_train)

# 예측
y_pred_gb = gb_model.predict(X_test)
y_proba_gb = gb_model.predict_proba(X_test)[:, 1]

# 평가
print("=" * 50)
print("Gradient Boosting 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_gb, target_names=['정상', '사기']))

---

## 2.4 트리 기반 모델 비교

In [ ]:
# 트리 기반 모델 비교
tree_models = {
    'Decision Tree': (y_pred_dt, y_proba_dt),
    'Random Forest': (y_pred_rf, y_proba_rf),
    'Gradient Boosting': (y_pred_gb, y_proba_gb)
}

tree_results = []
for name, (y_pred, y_proba) in tree_models.items():
    tree_results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'AUC': auc(*roc_curve(y_test, y_proba)[:2])
    })

df_tree_results = pd.DataFrame(tree_results)
print("트리 기반 모델 성능 비교:")
print(df_tree_results.round(4).to_string(index=False))

---

# Part 3: 앙상블 기법

---

## 3.1 Voting Classifier

### 원리
- 여러 모델의 예측을 결합
- Hard Voting: 다수결
- Soft Voting: 확률 평균

In [ ]:
# Voting Classifier (Soft Voting)
voting_model = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42))
    ],
    voting='soft'  # 확률 기반 결합
)

# 학습 (Logistic Regression은 스케일링 필요하지만 여기서는 원본 사용)
voting_model.fit(X_train, y_train)

# 예측
y_pred_voting = voting_model.predict(X_test)
y_proba_voting = voting_model.predict_proba(X_test)[:, 1]

print("=" * 50)
print("Voting Classifier (Soft) 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_voting, target_names=['정상', '사기']))

---

## 3.2 Bagging

### 원리
- Bootstrap Aggregating
- 부트스트랩 샘플로 여러 모델 학습 후 평균

In [ ]:
# Bagging with Decision Tree
bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=10),
    n_estimators=50,
    max_samples=0.8,
    max_features=0.8,
    n_jobs=-1,
    random_state=42
)
bagging_model.fit(X_train, y_train)

# 예측
y_pred_bagging = bagging_model.predict(X_test)
y_proba_bagging = bagging_model.predict_proba(X_test)[:, 1]

print("=" * 50)
print("Bagging 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_bagging, target_names=['정상', '사기']))

---

## 3.3 Boosting (AdaBoost)

### 원리
- 오분류 샘플에 가중치를 높여 순차 학습
- 약한 학습기를 강한 학습기로 결합

In [ ]:
# AdaBoost
ada_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)
ada_model.fit(X_train, y_train)

# 예측
y_pred_ada = ada_model.predict(X_test)
y_proba_ada = ada_model.predict_proba(X_test)[:, 1]

print("=" * 50)
print("AdaBoost 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_ada, target_names=['정상', '사기']))

---

## 3.4 Stacking

### 원리
- 기본 모델들의 예측을 새로운 특성으로 사용
- 메타 모델이 최종 예측 수행
- 가장 복잡하지만 높은 성능

In [ ]:
# Stacking Classifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
        ('svm', SVC(probability=True, random_state=42))
    ],
    final_estimator=LogisticRegression(random_state=42),
    cv=5
)
stacking_model.fit(X_train_scaled, y_train)

# 예측
y_pred_stacking = stacking_model.predict(X_test_scaled)
y_proba_stacking = stacking_model.predict_proba(X_test_scaled)[:, 1]

print("=" * 50)
print("Stacking 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_stacking, target_names=['정상', '사기']))

---

## 3.5 앙상블 기법 비교

In [ ]:
# 앙상블 기법 비교
ensemble_models = {
    'Voting': (y_pred_voting, y_proba_voting),
    'Bagging': (y_pred_bagging, y_proba_bagging),
    'AdaBoost': (y_pred_ada, y_proba_ada),
    'Stacking': (y_pred_stacking, y_proba_stacking)
}

ensemble_results = []
for name, (y_pred, y_proba) in ensemble_models.items():
    ensemble_results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'AUC': auc(*roc_curve(y_test, y_proba)[:2])
    })

df_ensemble_results = pd.DataFrame(ensemble_results)
print("앙상블 기법 성능 비교:")
print(df_ensemble_results.round(4).to_string(index=False))

---

# Part 4: 불균형 데이터 처리

---

## 4.1 클래스 불균형 문제 이해

In [ ]:
# 클래스 불균형 확인
class_counts = y_train.value_counts()
print("클래스 분포:")
print(f"  정상 (0): {class_counts[0]} ({class_counts[0]/len(y_train):.1%})")
print(f"  사기 (1): {class_counts[1]} ({class_counts[1]/len(y_train):.1%})")
print(f"\n불균형 비율: {class_counts[0] / class_counts[1]:.1f}:1")

# Plotly 파이 차트
fig = go.Figure(data=[go.Pie(
    labels=['정상', '사기'],
    values=[class_counts[0], class_counts[1]],
    hole=0.4,
    marker_colors=['steelblue', 'crimson']
)])
fig.update_layout(title="클래스 분포 (불균형)")
fig.show()

---

## 4.2 class_weight 조정

### 원리
- 소수 클래스에 높은 가중치 부여
- 모델이 소수 클래스 오분류에 더 큰 페널티

In [ ]:
# class_weight='balanced' 적용
lr_balanced = LogisticRegression(
    class_weight='balanced',  # 자동 가중치 조정
    random_state=42,
    max_iter=1000
)
lr_balanced.fit(X_train_scaled, y_train)

# 예측
y_pred_balanced = lr_balanced.predict(X_test_scaled)
y_proba_balanced = lr_balanced.predict_proba(X_test_scaled)[:, 1]

print("=" * 50)
print("Logistic Regression (class_weight='balanced') 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_balanced, target_names=['정상', '사기']))

# 기존 LR과 비교
print("\n기존 LR vs Balanced LR 비교:")
print(f"  기존 Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"  Balanced Recall: {recall_score(y_test, y_pred_balanced):.4f}")

In [ ]:
# Random Forest with class_weight
rf_balanced = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_balanced.fit(X_train, y_train)

y_pred_rf_balanced = rf_balanced.predict(X_test)
y_proba_rf_balanced = rf_balanced.predict_proba(X_test)[:, 1]

print("=" * 50)
print("Random Forest (class_weight='balanced') 결과")
print("=" * 50)
print(classification_report(y_test, y_pred_rf_balanced, target_names=['정상', '사기']))

---

## 4.3 SMOTE (오버샘플링)

### 원리
- Synthetic Minority Over-sampling Technique
- 소수 클래스의 합성 샘플 생성
- K-최근접 이웃 사이에 새 샘플 생성

In [ ]:
# SMOTE 설치 확인 및 적용
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    
    # SMOTE 적용
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)
    
    print("SMOTE 적용 전후 비교:")
    print(f"  적용 전: 정상 {sum(y_train==0)}, 사기 {sum(y_train==1)}")
    print(f"  적용 후: 정상 {sum(y_train_smote==0)}, 사기 {sum(y_train_smote==1)}")
    
    # SMOTE + Logistic Regression
    lr_smote = LogisticRegression(random_state=42, max_iter=1000)
    lr_smote.fit(X_train_smote, y_train_smote)
    
    y_pred_smote = lr_smote.predict(X_test_scaled)
    y_proba_smote = lr_smote.predict_proba(X_test_scaled)[:, 1]
    
    print("\n" + "=" * 50)
    print("SMOTE + Logistic Regression 결과")
    print("=" * 50)
    print(classification_report(y_test, y_pred_smote, target_names=['정상', '사기']))
    
except ImportError:
    print("imbalanced-learn 라이브러리가 설치되지 않았습니다.")
    print("설치: pip install imbalanced-learn")
    y_pred_smote = y_pred_balanced
    y_proba_smote = y_proba_balanced

---

## 4.4 Plotly로 ROC 곡선 시각화

In [ ]:
# 모든 모델의 ROC 곡선 비교
all_models = {
    'Logistic Regression': y_proba_lr,
    'Random Forest': y_proba_rf,
    'Gradient Boosting': y_proba_gb,
    'Stacking': y_proba_stacking,
    'LR (balanced)': y_proba_balanced,
    'RF (balanced)': y_proba_rf_balanced
}

fig = go.Figure()

colors = px.colors.qualitative.Set1

for i, (name, y_proba) in enumerate(all_models.items()):
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f"{name} (AUC={roc_auc:.3f})",
        line=dict(color=colors[i % len(colors)], width=2)
    ))

# 대각선 (랜덤 분류기)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random (AUC=0.5)',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title="ROC Curves - 모델 비교",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    template="plotly_white",
    legend=dict(x=0.6, y=0.1),
    width=800, height=600
)
fig.show()

---

## 4.5 Plotly로 PR 곡선 시각화

In [ ]:
# Precision-Recall 곡선 비교
fig = go.Figure()

for i, (name, y_proba) in enumerate(all_models.items()):
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    
    fig.add_trace(go.Scatter(
        x=recall, y=precision,
        mode='lines',
        name=f"{name} (AP={ap:.3f})",
        line=dict(color=colors[i % len(colors)], width=2)
    ))

# 베이스라인 (랜덤)
baseline = y_test.mean()
fig.add_hline(y=baseline, line_dash="dash", line_color="gray",
              annotation_text=f"Baseline ({baseline:.2%})")

fig.update_layout(
    title="Precision-Recall Curves - 모델 비교",
    xaxis_title="Recall",
    yaxis_title="Precision",
    template="plotly_white",
    legend=dict(x=0.02, y=0.3),
    width=800, height=600
)
fig.show()

print("\n불균형 데이터에서는 PR 곡선이 ROC보다 더 유용합니다!")
print("AP (Average Precision)가 높을수록 좋은 모델입니다.")

---

## 4.6 최종 모델 성능 종합 비교

In [ ]:
# 전체 모델 성능 비교
final_models = {
    'Logistic Regression': (y_pred_lr, y_proba_lr),
    'KNN': (y_pred_knn, y_proba_knn),
    'Naive Bayes': (y_pred_nb, y_proba_nb),
    'SVM': (y_pred_svm, y_proba_svm),
    'Decision Tree': (y_pred_dt, y_proba_dt),
    'Random Forest': (y_pred_rf, y_proba_rf),
    'Gradient Boosting': (y_pred_gb, y_proba_gb),
    'Voting': (y_pred_voting, y_proba_voting),
    'Bagging': (y_pred_bagging, y_proba_bagging),
    'AdaBoost': (y_pred_ada, y_proba_ada),
    'Stacking': (y_pred_stacking, y_proba_stacking),
    'LR (balanced)': (y_pred_balanced, y_proba_balanced),
    'RF (balanced)': (y_pred_rf_balanced, y_proba_rf_balanced)
}

final_results = []
for name, (y_pred, y_proba) in final_models.items():
    final_results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'AUC': auc(*roc_curve(y_test, y_proba)[:2]),
        'AP': average_precision_score(y_test, y_proba)
    })

df_final = pd.DataFrame(final_results).sort_values('F1', ascending=False)
print("전체 모델 성능 비교 (F1 기준 정렬):")
print(df_final.round(4).to_string(index=False))

In [ ]:
# 최종 성능 비교 히트맵
df_heatmap = df_final.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1', 'AUC', 'AP']]

fig = go.Figure(data=go.Heatmap(
    z=df_heatmap.values,
    x=df_heatmap.columns,
    y=df_heatmap.index,
    colorscale='RdYlGn',
    text=df_heatmap.values.round(3),
    texttemplate="%{text}",
    textfont={"size": 10}
))

fig.update_layout(
    title="모델별 성능 히트맵",
    xaxis_title="Metric",
    yaxis_title="Model",
    width=800, height=600
)
fig.show()

---

## 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. Logistic Regression 기본 훈련 ⭐

**문제**: 아래 데이터로 Logistic Regression 모델을 훈련하고 정확도를 출력하세요.

```python
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target
```

**기대 결과**: 테스트 정확도 출력 (test_size=0.2)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X, y = data.data, data.target

# 여기에 코드를 작성하세요


### Q2. KNN 모델 K값 실험 ⭐

**문제**: K=3, K=5, K=7로 KNN 모델을 훈련하고 각각의 정확도를 비교하세요.

**기대 결과**: 각 K값에 대한 정확도 출력

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 위의 breast_cancer 데이터 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 여기에 코드를 작성하세요


### Q3. Decision Tree 특성 중요도 추출 ⭐⭐

**문제**: Decision Tree를 훈련하고 상위 5개 중요 특성을 출력하세요.

**기대 결과**: 특성 이름과 중요도 (내림차순)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

feature_names = data.feature_names

# 여기에 코드를 작성하세요


### Q4. Random Forest vs Decision Tree 비교 ⭐⭐

**문제**: Random Forest (n_estimators=100)와 Decision Tree의 성능을 비교하세요.

**기대 결과**: 두 모델의 정확도, F1 Score 비교

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# 여기에 코드를 작성하세요


### Q5. Confusion Matrix 계산 ⭐⭐

**문제**: 사기 탐지 데이터에서 Random Forest의 Confusion Matrix를 출력하고 TP, FP, TN, FN을 해석하세요.

**기대 결과**: Confusion Matrix + 각 값의 의미 설명

In [ ]:
from sklearn.metrics import confusion_matrix

# 앞서 훈련한 rf_model 사용

# 여기에 코드를 작성하세요


### Q6. Voting Classifier 구현 ⭐⭐⭐

**문제**: LogisticRegression, RandomForest, GradientBoosting을 결합한 Soft Voting Classifier를 구현하세요.

**기대 결과**: Voting Classifier의 정확도와 개별 모델 정확도 비교

In [ ]:
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier

# 여기에 코드를 작성하세요


### Q7. class_weight 효과 분석 ⭐⭐⭐

**문제**: 사기 탐지 데이터에서 class_weight 적용 전후의 Recall을 비교하세요.

**기대 결과**: class_weight 없음 vs 'balanced'의 Recall 비교

In [ ]:
from sklearn.metrics import recall_score

# 여기에 코드를 작성하세요


### Q8. ROC 곡선 Plotly 시각화 ⭐⭐⭐

**문제**: Logistic Regression과 Random Forest의 ROC 곡선을 Plotly로 시각화하세요.

**기대 결과**: 두 모델의 ROC 곡선 + AUC 값 표시

In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

# 여기에 코드를 작성하세요


### Q9. Stacking Ensemble 구현 ⭐⭐⭐⭐

**문제**: 3개의 기본 모델과 메타 모델을 사용한 Stacking Classifier를 구현하세요.

조건:
- 기본 모델: KNN, Decision Tree, SVM
- 메타 모델: Logistic Regression

**기대 결과**: Stacking 모델의 정확도와 classification_report

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

# 여기에 코드를 작성하세요


### Q10. 종합: 최적 분류 모델 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 사기 탐지 데이터에 대해 최적의 분류 파이프라인을 구축하세요.

요구사항:
1. 최소 3개 모델 비교
2. 불균형 처리 기법 적용 (class_weight 또는 SMOTE)
3. Plotly로 ROC 곡선과 PR 곡선 시각화
4. 최종 추천 모델 선정 및 이유 설명

**기대 결과**: 성능 비교표 + 시각화 + 추천 모델 설명

In [ ]:
# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기본 분류기 핵심 요약

| 모델 | 장점 | 단점 | 사용 시점 |
|------|------|------|----------|
| Logistic Regression | 해석 가능, 빠름 | 비선형 패턴 약함 | 베이스라인, 해석 중요 |
| KNN | 단순, 비모수 | 고차원 취약, 느림 | 소규모 데이터 |
| Naive Bayes | 매우 빠름 | 독립 가정 | 텍스트 분류 |
| SVM | 고차원 효과적 | 튜닝 필요, 느림 | 중소규모, 복잡한 경계 |

### Part 2: 트리 기반 모델 핵심 요약

| 모델 | 특징 | 과적합 방지 | 사용 시점 |
|------|------|------------|----------|
| Decision Tree | 해석 가능, 규칙 추출 | max_depth, min_samples | 규칙 기반 의사결정 |
| Random Forest | 안정적, 병렬 처리 | 배깅 앙상블 | 대부분의 정형 데이터 |
| Gradient Boosting | 높은 성능 | 학습률, 정규화 | 최고 성능 필요시 |

### Part 3: 앙상블 기법 핵심 요약

| 기법 | 원리 | 장점 | 적용 |
|------|------|------|------|
| Voting | 다수결/확률 평균 | 단순, 효과적 | 다양한 모델 결합 |
| Bagging | 부트스트랩 앙상블 | 분산 감소 | 과적합 방지 |
| Boosting | 순차 오차 학습 | 편향 감소 | 높은 성능 |
| Stacking | 메타 모델 결합 | 최고 성능 | Kaggle 대회 |

### Part 4: 불균형 데이터 핵심 요약

| 기법 | 방식 | 장점 | 주의점 |
|------|------|------|--------|
| class_weight | 가중치 조정 | 간단, 빠름 | 모델 내장 필요 |
| SMOTE | 오버샘플링 | 효과적 | 노이즈 생성 가능 |
| 언더샘플링 | 다수 클래스 축소 | 빠름 | 정보 손실 |
| 임계값 조정 | 결정 경계 이동 | 유연함 | 최적 임계값 탐색 |

### 실무 팁

1. **베이스라인 먼저**: Logistic Regression으로 시작, 점진적 복잡화
2. **트리 기반 모델 추천**: Random Forest/GradientBoosting이 대부분 잘 작동
3. **앙상블은 마지막**: 개별 모델 최적화 후 앙상블 시도
4. **불균형 = Recall 중시**: 사기/질병 탐지에서 FN 비용 > FP 비용
5. **PR 곡선 사용**: 불균형 데이터에서 ROC보다 PR 곡선이 유용
6. **교차 검증 필수**: 단일 split 결과 신뢰 금지
7. **특성 중요도 활용**: 모델 해석 및 특성 선택에 활용